# Generate the files

## Requirements

[rustup](https://rustup.rs/)
[python3](https://www.python.org/downloads/)

Maybe virtual env?

## Step by Step
We need to locate the directory where the replays are, in my case I dual-boot and mount windows to `/mnt/windows`

The final step takes around 32 seconds to process 3600 Replay files. This means about 110 replays per-second on a 16 Core machine.
```bash
$ git clone https://github.com/sebosp/s2protocol-rs
$ cd s2protocol-rs
$ mkdir ipcs/
$ cargo run --features arrow,syntax -r -- -v error --timing --source /home/seb/SCReplaysOnNVMe --output /home/seb/git/s2protocol-rs/ipcs/ write-arrow-ipc --process-max-files 10000000
36752 files have valid init data, processing...
Total time: 234.08368818s
$ du -sh ipcs/*
1.8G    ipcs/cmd_target_point.ipc
750M    ipcs/cmd_target_unit.ipc
9.7M    ipcs/details.ipc
112M    ipcs/lobby_init_data.ipc
2.1G    ipcs/stats.ipc
9.3G    ipcs/unit_born.ipc
5.5G    ipcs/unit_died.ipc
172M    ipcs/upgrades.ipc
36M     ipcs/user_init_data.ipc
$ cd ../s2-polars-data-analysis/
$ pip install -r requirements.txt
$ jupyter lab --notebook-dir=jupyter_notebooks/
# Open the URL explained in the terminal, this will open this notebook for interacting with the data.
```


In [43]:
# import datashader as ds
import plotly.express as px
import polars as pl

pl.Config.set_tbl_width_chars(256)
pl.Config.set_fmt_str_lengths(256)
pl.Config.set_tbl_rows(24)

# Location of the IPC generated files from above.
ipc_dir = "/home/seb/git/s2protocol-rs/ipcs"

# Filter out these:
# Beacon*: Similiar to above, default targets of hatcheries and buildings
#          points to Beacons, clicks on the map, or actions when teams play
#          together and send attack/defend points on the map.
#          For now we'll avoid them.

unit_born_df = pl.scan_ipc(f"{ipc_dir}/unit_born.ipc")
# .filter((~pl.col("unit_type_name").str.starts_with("Beacon")))
unit_died_df = pl.scan_ipc(f"{ipc_dir}/unit_died.ipc")
# .filter((~pl.col("unit_died_name").str.starts_with("Beacon")))
stats_df = pl.scan_ipc(f"{ipc_dir}/stats.ipc")
upgrades_df = pl.scan_ipc(f"{ipc_dir}/upgrades.ipc")
lobby_slot_init_data_df = pl.scan_ipc(f"{ipc_dir}/lobby_init_data.ipc")
user_init_data_df = pl.scan_ipc(f"{ipc_dir}/user_init_data.ipc")
details_df = pl.scan_ipc(f"{ipc_dir}/details.ipc")
cmd_target_point_df = pl.scan_ipc(f"{ipc_dir}/cmd_target_point.ipc")
cmd_target_unit_df = pl.scan_ipc(f"{ipc_dir}/cmd_target_unit.ipc")

# Find a game:
lobby_slot_init_data_df.filter(
    [
        pl.col("ext_fs_file_name").str.contains("Grand Finals")
        & pl.col("ext_fs_file_name").str.contains("2025")
        & pl.col("observe").eq(0)
    ]
).collect()

ext_fs_id,ext_fs_sha256,ext_fs_file_name,control,user_id,team_id,observe,working_set_slot_id,map_size_x,map_size_y
u64,str,str,i64,i64,i64,u8,u8,u8,u8
16590,"""035622859a854c30852e566a11e0600aa1041fc7b327254146e04469362a4786""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game3 - Classic vs Serral - Ultralove.SC2Replay""",2,2,1,0,11,184,184
16590,"""035622859a854c30852e566a11e0600aa1041fc7b327254146e04469362a4786""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game3 - Classic vs Serral - Ultralove.SC2Replay""",2,3,0,0,12,184,184
16591,"""e79a9db4ae3fea38c557cb798e8bd712220f4517b37b3cf51cc5f12e5114af64""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game4 - Classic vs Serral - Pylon.SC2Replay""",2,2,0,0,11,248,250
16591,"""e79a9db4ae3fea38c557cb798e8bd712220f4517b37b3cf51cc5f12e5114af64""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game4 - Classic vs Serral - Pylon.SC2Replay""",2,5,1,0,14,248,250
16592,"""99239a9156f3d0102c5f938c04e9a5ab4b9b1733d67dd12335749b18a79d9907""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game6 - Classic vs Serral - Torches.SC2Replay""",2,2,1,0,11,160,210
16592,"""99239a9156f3d0102c5f938c04e9a5ab4b9b1733d67dd12335749b18a79d9907""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game6 - Classic vs Serral - Torches.SC2Replay""",2,3,0,0,12,160,210
16593,"""4d15cf43425b9bdb78770dbce0daaa7f03e3d987f565cb783ba42a044ffca789""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game7 - Classic vs Serral - Magannatha.SC2Replay""",2,4,0,0,13,168,170
16593,"""4d15cf43425b9bdb78770dbce0daaa7f03e3d987f565cb783ba42a044ffca789""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game7 - Classic vs Serral - Magannatha.SC2Replay""",2,5,1,0,14,168,170
16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184


In [44]:
chosen_fs_id = 16594  # One of the above

In [45]:
lobby_slot_init_data_df.collect_schema()

Schema([('ext_fs_id', UInt64),
        ('ext_fs_sha256', String),
        ('ext_fs_file_name', String),
        ('control', Int64),
        ('user_id', Int64),
        ('team_id', Int64),
        ('observe', UInt8),
        ('working_set_slot_id', UInt8),
        ('map_size_x', UInt8),
        ('map_size_y', UInt8)])

In [46]:
lobby_slot_init_data_df.describe()

statistic,ext_fs_id,ext_fs_sha256,ext_fs_file_name,control,user_id,team_id,observe,working_set_slot_id,map_size_x,map_size_y
str,f64,str,str,f64,f64,f64,f64,f64,f64,f64
"""count""",585914.0,"""585914""","""585914""",585914.0,150806.0,585914.0,585914.0,585904.0,585914.0,585914.0
"""null_count""",0.0,"""0""","""0""",0.0,435108.0,0.0,0.0,10.0,0.0,0.0
"""mean""",28656.471163,null,null,0.941997,2.540602,0.112332,0.891124,7.513982,186.989913,187.694682
"""std""",14873.434007,null,null,0.868435,2.579175,0.343804,0.437713,4.610135,33.331622,33.721996
"""min""",0.0,"""00002a9dc36528452ad1938d035d16186b31390e6faef0487a0468214d644b34""","""/home/seb/SCReplaysOnNVMe/2023_07-ESL_SC2_Masters_Winter_2023_Finals/1 - Winners Stage/A1 - UB Ro8 #1 - GuMiho vs ShoWTimE/20230616 - Game 2 - ShoWTimE vs GuMiho - PvT - Hecate.SC2Replay""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",15511.0,null,null,0.0,1.0,0.0,1.0,4.0,168.0,176.0
"""50%""",28338.0,null,null,1.0,2.0,0.0,1.0,8.0,192.0,186.0
"""75%""",41541.0,null,null,2.0,4.0,0.0,1.0,12.0,200.0,208.0
"""max""",54323.0,"""fffe9e9afdd6676125f0204935343add1386f924147125e050bdea4838f60d9f""","""/home/seb/SCReplaysOnNVMe/[2024 GSL S2] Ro.8 Group B Match5/2SET [2024 GSL S2] Ro.8 Group B Match5.SC2Replay""",3.0,15.0,14.0,2.0,15.0,248.0,250.0


In [47]:
lobby_slot_init_data_df.select(pl.col("user_id").value_counts(sort=True)).collect()
# - observe 0, 1, 2 --- 0 = player
# - control 0,1,2,3 --- unsure, most are 1.
# - user_id null, 0, 1..15 --- unsure, most are null (155Ks), then 0, 12Ks = matches number of files.
# - team_id 0, 1 --- seems for players
# - working_set_slot_id 0..15 --- unsure (are these maybe the slots of 16 players that can be in a lobby?
#   For example active players can be 12, but doesn't match the event player ids
# - map_size_x: 0? 1..16 possible values of size_x
# - map_size_y: 0? 1.. many more than 16 possible values? doesn't match size_x hmm18780

user_id
struct[2]
"{null,435108}"
"{0,36752}"
"{1,34012}"
"{2,18672}"
"{3,17298}"
"{4,13350}"
"{5,10190}"
"{6,7917}"
"{7,4209}"


In [48]:
lobby_slot_init_data_df.filter(
    [pl.col("ext_fs_id").eq(chosen_fs_id) & pl.col("observe").eq(0)]
).collect()

ext_fs_id,ext_fs_sha256,ext_fs_file_name,control,user_id,team_id,observe,working_set_slot_id,map_size_x,map_size_y
u64,str,str,i64,i64,i64,u8,u8,u8,u8
16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,3,1,0,12,160,184


In [49]:
details_df.collect_schema()

Schema([('player_name', String),
        ('player_toon_region', UInt8),
        ('player_toon_program_id', UInt32),
        ('player_toon_realm', UInt32),
        ('player_toon_id', UInt64),
        ('player_race', String),
        ('player_color_a', UInt8),
        ('player_color_r', UInt8),
        ('player_color_g', UInt8),
        ('player_color_b', UInt8),
        ('player_control', UInt8),
        ('player_team_id', UInt8),
        ('player_observe', UInt8),
        ('player_result', UInt8),
        ('player_working_set_slot_id', UInt8),
        ('player_hero', String),
        ('title', String),
        ('is_blizzard_map', Boolean),
        ('time_utc', Int64),
        ('time_local_offset', Int64),
        ('ext_fs_id', UInt64),
        ('ext_datetime', Datetime(time_unit='ns', time_zone=None))])

In [50]:
details_df.describe()

statistic,player_name,player_toon_region,player_toon_program_id,player_toon_realm,player_toon_id,player_race,player_color_a,player_color_r,player_color_g,player_color_b,player_control,player_team_id,player_observe,player_result,player_working_set_slot_id,player_hero,title,is_blizzard_map,time_utc,time_local_offset,ext_fs_id,ext_datetime
str,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,str
"""count""","""91303""",91303.0,91303.0,91303.0,91303.0,"""91303""",91303.0,91303.0,91303.0,91303.0,91303.0,91303.0,91303.0,91303.0,90720.0,"""91303""","""91303""",91303.0,91303.0,91303.0,91303.0,"""91303"""
"""null_count""","""0""",0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,583.0,"""0""","""0""",0.0,0.0,0.0,0.0,"""0"""
"""mean""",null,1.523981,18093.607746,0.896937,5.7489e6,null,254.950768,91.923113,56.68897,142.136173,2.150389,0.50811,0.0,1.369243,3.412831,null,null,0.781431,1.3305e17,2.5669e10,27766.430764,"""2022-08-09 13:05:21.304537"""
"""std""",null,1.412723,7614.437047,0.432697,4.6587e6,null,1.234413,89.723479,52.179126,107.963496,0.357455,0.584962,0.0,0.634426,4.646266,null,null,null,4.3520e14,1.4609e11,14648.532064,null
"""min""","""&lt;0001&gt;<sp/>VOODOO""",0.0,0.0,0.0,0.0,"""""",224.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,"""""","""$$$Fastest Map$$$""",0.0,1.3240e17,-3.6000e11,0.0,"""2020-07-28 02:30:03.466019"""
"""25%""",null,1.0,21298.0,1.0,1.381873e6,null,255.0,0.0,20.0,30.0,2.0,0.0,0.0,1.0,0.0,null,null,null,1.3266e17,3.6000e10,14652.0,"""2021-05-25 16:23:25.967399"""
"""50%""",null,2.0,21298.0,1.0,5.179818e6,null,255.0,84.0,66.0,129.0,2.0,0.0,0.0,1.0,1.0,null,null,null,1.3301e17,3.6000e10,26400.0,"""2022-07-01 17:42:24.452301"""
"""75%""",null,2.0,21298.0,1.0,9.939056e6,null,255.0,180.0,66.0,255.0,2.0,1.0,0.0,2.0,3.0,null,null,null,1.3335e17,7.2000e10,40354.0,"""2023-08-02 13:47:02.582020"""
"""max""","""호에엥""",98.0,21298.0,2.0,2.0889428e7,"""프로토스""",255.0,255.0,255.0,255.0,3.0,14.0,0.0,3.0,15.0,"""""","""해비테이션 스테이션 - 래더""",1.0,1.3399e17,4.6800e11,54323.0,"""2025-08-04 06:44:28.415091"""


In [51]:
details_df.filter(pl.col("player_toon_id") != 0).select(
    pl.col("player_name").value_counts(sort=True)
).collect()

# player_name contains the clan, clan also in user_inuser_init_data_df separate from name, but no user_id/player_id there. Can be cleaned with pl.col("player_name").str.split(by="<sp/>").list.last().alias("player_name")
# player_toon_id may be 0 for A.I. game practice.
# player_race is Protoss, Terran, but also has hanzi (神族), korean? (테란) and strings like "Primal Zerg" or "Terraner" or ""
# player_team_id is either 0 or 1
# player_observe is always 0
# player_result can be 1, 2,0 and 3
#               1 and 2 are win,loss (not sure which is which).
#               0 is probably undecided (or maybe teh status of observer?)
#               3 is probably tie (there are only 50 games with this status in this 12K replay dataset.
# player_working_set_slot_id null, 0..15
#                            not the same as lobby_slot_init_data_df working_set_slot_id
# player_hero always ""
# title is the name of the map
# is_blizzard_map seems filled
# time_utc is that weird "epoch" after 2000 or something
# time_local_offset is +- in millis (or nanos?)

player_name
struct[2]
"{""&lt;Mealen&gt;<sp/>Doombringer"",3226}"
"{""&lt;chezs&gt;<sp/>Sazed"",2406}"
"{""Sazed"",1880}"
"{""Massa"",1661}"
"{""qqq"",1374}"
"{""&lt;ƖIıIƖ&gt;<sp/>Sazed"",744}"
"{""Serral"",679}"
"{""Reynor"",669}"
"{""ShoWTimE"",658}"


In [52]:
# Remove the clan from the player_name
# the clan and player are separated already in the user init data
# but I don't see a way to tie it to the lobby_state user data).
details_df = details_df.with_columns(
    pl.col("player_name").str.split(by="<sp/>").list.last().alias("player_name")
)
details_df.filter(pl.col("player_toon_id") != 0).select(
    pl.col("player_name").value_counts(sort=True)
).collect()

player_name
struct[2]
"{""Sazed"",5055}"
"{""Doombringer"",3226}"
"{""Massa"",1661}"
"{""qqq"",1374}"
"{""Reynor"",1017}"
"{""Serral"",993}"
"{""HeroMarine"",827}"
"{""ShoWTimE"",808}"
"{""Lambo"",770}"


In [53]:
details_lobby_df = details_df.filter([pl.col("ext_fs_id").eq(chosen_fs_id)]).join(
    lobby_slot_init_data_df.filter([pl.col("ext_fs_id").eq(chosen_fs_id)]).with_columns(
        pl.col("working_set_slot_id").alias("player_working_set_slot_id")
    ),
    on="player_working_set_slot_id",
)
details_lobby_df.collect_schema()

Schema([('player_name', String),
        ('player_toon_region', UInt8),
        ('player_toon_program_id', UInt32),
        ('player_toon_realm', UInt32),
        ('player_toon_id', UInt64),
        ('player_race', String),
        ('player_color_a', UInt8),
        ('player_color_r', UInt8),
        ('player_color_g', UInt8),
        ('player_color_b', UInt8),
        ('player_control', UInt8),
        ('player_team_id', UInt8),
        ('player_observe', UInt8),
        ('player_result', UInt8),
        ('player_working_set_slot_id', UInt8),
        ('player_hero', String),
        ('title', String),
        ('is_blizzard_map', Boolean),
        ('time_utc', Int64),
        ('time_local_offset', Int64),
        ('ext_fs_id', UInt64),
        ('ext_datetime', Datetime(time_unit='ns', time_zone=None)),
        ('ext_fs_id_right', UInt64),
        ('ext_fs_sha256', String),
        ('ext_fs_file_name', String),
        ('control', Int64),
        ('user_id', Int64),
        ('team_id', 

In [54]:
upgrades_df.collect_schema()

Schema([('player_id', UInt8),
        ('name', String),
        ('count', Int32),
        ('ext_replay_loop', Int64),
        ('ext_replay_seconds', UInt32),
        ('ext_fs_id', UInt64)])

In [55]:
upgrades_df.describe()

statistic,player_id,name,count,ext_replay_loop,ext_replay_seconds,ext_fs_id
str,f64,str,f64,f64,f64,f64
"""count""",3.07452e6,"""3074520""",3.07452e6,3.07452e6,3.07452e6,3.07452e6
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0
"""mean""",3.136683,null,0.681878,12500.384307,608.293145,30582.066743
"""std""",3.056639,null,1.406891,19045.01724,927.225831,13511.01449
"""min""",0.0,"""05EnergyRegen""",-64.0,0.0,0.0,0.0
"""25%""",1.0,null,0.0,0.0,0.0,18899.0
"""50%""",2.0,null,1.0,7121.0,346.0,33954.0
"""75%""",4.0,null,1.0,18643.0,907.0,39208.0
"""max""",15.0,"""zerglingmovementspeed""",127.0,735294.0,35802.0,54323.0


In [56]:
upgrades_df.select(pl.col("name").value_counts(sort=True)).collect()
# player_id is 0..15
#             0 is only 20 occurrences and it's from campaign
#             From 1 it's actual players.
# name: the name of the upgrade, many are like SprayZerg/Protoss or RewardDance<Unit>
# I checked both  the s2protocol python module and debugged this one and they both are missing most of the upgardes...
# Maybe after a specific version it comes in different event type?

name
struct[2]
"{""SprayZerg"",389339}"
"{""SprayTerran"",236029}"
"{""SprayProtoss"",227133}"
"{""RewardDanceMule"",45239}"
"{""zerglingmovementspeed"",45118}"
"{""RewardDanceOverlord"",44519}"
"{""RewardDanceRoach"",42632}"
"{""RewardDanceViking"",42388}"
"{""RewardDanceStalker"",41803}"


In [57]:
# Clean up a little bit the upgrades:
upgrades_df = upgrades_df.filter(
    [
        ~pl.col("name").str.contains("Reward")
        & ~pl.col("name").str.contains("Spray")
        & ~pl.col("name").str.contains("GameHeart")
    ]
)
upgrades_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).collect()

player_id,name,count,ext_replay_loop,ext_replay_seconds,ext_fs_id
u8,str,i32,i64,u32,u64
1,"""zerglingmovementspeed""",1,6584,320,16594
2,"""WarpGateResearch""",1,7728,376,16594
2,"""BlinkTech""",1,14609,711,16594
1,"""GlialReconstitution""",1,14643,712,16594
2,"""ProtossGroundWeaponsLevel1""",1,14921,726,16594
2,"""PsiStormTech""",1,14981,729,16594
1,"""overlordspeed""",1,15041,732,16594
1,"""ZergMissileWeaponsLevel1""",1,16403,798,16594


In [58]:
upgrades_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).join(
    details_lobby_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).with_columns(
        pl.col("user_id").sub(1).alias("player_id")
    ),
    on=["player_id", "ext_fs_id"],
).collect()

player_id,name,count,ext_replay_loop,ext_replay_seconds,ext_fs_id,player_name,player_toon_region,player_toon_program_id,player_toon_realm,player_toon_id,player_race,player_color_a,player_color_r,player_color_g,player_color_b,player_control,player_team_id,player_observe,player_result,player_working_set_slot_id,player_hero,title,is_blizzard_map,time_utc,time_local_offset,ext_datetime,ext_fs_id_right,ext_fs_sha256,ext_fs_file_name,control,user_id,team_id,observe,working_set_slot_id,map_size_x,map_size_y
u8,str,i32,i64,u32,u64,str,u8,u32,u32,u64,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,str,bool,i64,i64,datetime[ns],u64,str,str,i64,i64,i64,u8,u8,u8,u8
1,"""zerglingmovementspeed""",1,6584,320,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
2,"""WarpGateResearch""",1,7728,376,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,3,1,0,12,160,184
2,"""BlinkTech""",1,14609,711,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,3,1,0,12,160,184
1,"""GlialReconstitution""",1,14643,712,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
2,"""ProtossGroundWeaponsLevel1""",1,14921,726,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,3,1,0,12,160,184
2,"""PsiStormTech""",1,14981,729,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,3,1,0,12,160,184
1,"""overlordspeed""",1,15041,732,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
1,"""ZergMissileWeaponsLevel1""",1,16403,798,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",

In [59]:
stats_df.collect_schema()

Schema([('player_id', UInt8),
        ('minerals_current', Int32),
        ('vespene_current', Int32),
        ('minerals_collection_rate', Int32),
        ('vespene_collection_rate', Int32),
        ('workers_active_count', Int32),
        ('minerals_used_in_progress_army', Int32),
        ('minerals_used_in_progress_economy', Int32),
        ('minerals_used_in_progress_technology', Int32),
        ('vespene_used_in_progress_army', Int32),
        ('vespene_used_in_progress_economy', Int32),
        ('vespene_used_in_progress_technology', Int32),
        ('minerals_used_current_army', Int32),
        ('minerals_used_current_economy', Int32),
        ('minerals_used_current_technology', Int32),
        ('vespene_used_current_army', Int32),
        ('vespene_used_current_economy', Int32),
        ('vespene_used_current_technology', Int32),
        ('minerals_lost_army', Int32),
        ('minerals_lost_economy', Int32),
        ('minerals_lost_technology', Int32),
        ('vespene_lost_

In [60]:
stats_df.describe()

statistic,player_id,minerals_current,vespene_current,minerals_collection_rate,vespene_collection_rate,workers_active_count,minerals_used_in_progress_army,minerals_used_in_progress_economy,minerals_used_in_progress_technology,vespene_used_in_progress_army,vespene_used_in_progress_economy,vespene_used_in_progress_technology,minerals_used_current_army,minerals_used_current_economy,minerals_used_current_technology,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,minerals_lost_army,minerals_lost_economy,minerals_lost_technology,vespene_lost_army,vespene_lost_economy,vespene_lost_technology,minerals_killed_army,minerals_killed_economy,minerals_killed_technology,vespene_killed_army,vespene_killed_economy,vespene_killed_technology,food_used,food_made,minerals_used_active_forces,vespene_used_active_forces,minerals_friendly_fire_army,minerals_friendly_fire_economy,minerals_friendly_fire_technology,vespene_friendly_fire_army,vespene_friendly_fire_economy,vespene_friendly_fire_technology,ext_replay_loop,ext_replay_seconds,ext_fs_id
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7,1.0867638e7
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",2.040324,7161.093621,1540.784684,1509.381282,485.889025,38.864301,259.623342,206.994603,108.828687,127.791416,2.360511,30.482875,2583.153929,4680.445414,2033.827134,1141.224279,120.237839,423.274576,5956.806849,1258.313263,422.676062,2582.784966,37.504705,45.684042,8137.794204,1320.571318,536.584637,3785.362793,28.504384,54.565683,86.542436,122.145925,2744.362845,1136.863703,225.382313,47.959367,26.294944,101.8621,0.464176,6.076989,23702.594847,1153.570983,25998.632221
"""std""",1.360996,1.0251e6,11413.920231,1037.193691,531.145462,23.136313,420.724178,260.74912,186.148218,259.105211,100.749751,78.619912,97231.920522,5300.120144,10031.154468,2702.841842,4006.203723,1333.786838,85056.322284,3865.204279,2854.684413,8518.814939,1988.633847,801.964369,152780.194088,5875.302866,21784.665972,75294.089175,1591.862899,910.432084,63.948118,233.455406,97229.617961,2698.848045,1004.521885,175.743325,156.282546,668.389535,28.137779,104.566453,35302.389555,1718.943469,14450.697439
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,-12.0,-450.0,0.0,0.0,0.0,-5e7,-371175.0,-69300.0,-4020.0,-294.0,-690.0,-800.0,-1344.0,-1475.0,-900.0,-90.0,-900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5e7,-1700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",1.0,125.0,65.0,811.0,134.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,2100.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,40.0,222.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6986.0,340.0,13790.0
"""50%""",2.0,324.0,230.0,1399.0,335.0,36.0,125.0,100.0,0.0,0.0,0.0,0.0,1250.0,4200.0,1100.0,375.0,0.0,275.0,400.0,50.0,0.0,50.0,0.0,0.0,400.0,75.0,0.0,50.0,0.0,0.0,72.0,101.0,1300.0,375.0,0.0,0.0,0.0,0.0,0.0,0.0,14648.0,713.0,22820.0
"""75%""",2.0,1035.0,715.0,2172.0,694.0,58.0,375.0,400.0,150.0,175.0,0.0,0.0,3450.0,6725.0,2350.0,1550.0,0.0,650.0,4300.0,825.0,50.0,1350.0,0.0,0.0,3950.0,800.0,125.0,1275.0,0.0,0.0,138.0,186.0,3600.0,1525.0,0.0,0.0,0.0,0.0,0.0,0.0,25466.0,1239.0,38324.0
"""max""",15.0,2.68435651e8,1.323338e6,159028.0,72794.0,200.0,90600.0,90000.0,35950.0,45000.0,90000.0,1850

In [61]:
stats_df.select(
    pl.col("minerals_used_in_progress_economy").value_counts(sort=True)
).collect()
# player_id From 0..14, mostly 0 and 1, not sure which matches with working_set_slot_id
# minerals_current self explanatory at a given ext_replay_loop for a specific user_id
# vespene_current same as minerals_current
# minerals_collection_rate unsure, seems to be from 0 to 14Ks, 1566 mean. maybe per minute?
# vespene_collection_rate from 0 to 4Ks, 549 mean, as with minerals_collection_rate?
# workers_active_count from 0 to 200 (oddly)
# ------ Following fields are "resource" type:
# ------ - Minerals
# ------ - Vespene
# ------ And for each resource, one stored field per "bucket"
# ------ - Army
# ------ - Economy
# ------ - Technology
# <resource>_used_in_progress_<bucket>
# <resource>_used_current_<bucket>
# <resource>_lost_<bucket>
# <resource>_killed_<bucket>
# <resource>_friendly_fire_<bucket>
# ------ And then two more:
# <resource>_used_active_forces

minerals_used_in_progress_economy
struct[2]
"{0,3603803}"
"{100,959676}"
"{50,847404}"
"{150,754554}"
"{400,648313}"
"{200,493650}"
"{450,456594}"
"{350,456481}"
"{500,378672}"


In [62]:
stats_df.filter(
    [pl.col("ext_fs_id").eq(chosen_fs_id) & pl.col("ext_replay_loop").eq(1)]
).join(
    details_lobby_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).with_columns(
        pl.col("user_id").sub(1).alias("player_id")
    ),
    on=["player_id", "ext_fs_id"],
).collect()

player_id,minerals_current,vespene_current,minerals_collection_rate,vespene_collection_rate,workers_active_count,minerals_used_in_progress_army,minerals_used_in_progress_economy,minerals_used_in_progress_technology,vespene_used_in_progress_army,vespene_used_in_progress_economy,vespene_used_in_progress_technology,minerals_used_current_army,minerals_used_current_economy,minerals_used_current_technology,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,minerals_lost_army,minerals_lost_economy,minerals_lost_technology,vespene_lost_army,vespene_lost_economy,vespene_lost_technology,minerals_killed_army,minerals_killed_economy,minerals_killed_technology,vespene_killed_army,vespene_killed_economy,vespene_killed_technology,food_used,food_made,minerals_used_active_forces,vespene_used_active_forces,minerals_friendly_fire_army,minerals_friendly_fire_economy,minerals_friendly_fire_technology,vespene_friendly_fire_army,vespene_friendly_fire_economy,vespene_friendly_fire_technology,ext_replay_loop,ext_replay_seconds,ext_fs_id,player_name,player_toon_region,player_toon_program_id,player_toon_realm,player_toon_id,player_race,player_color_a,player_color_r,player_color_g,player_color_b,player_control,player_team_id,player_observe,player_result,player_working_set_slot_id,player_hero,title,is_blizzard_map,time_utc,time_local_offset,ext_datetime,ext_fs_id_right,ext_fs_sha256,ext_fs_file_name,control,user_id,team_id,observe,working_set_slot_id,map_size_x,map_size_y
u8,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i64,u32,u64,str,u8,u32,u32,u64,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,str,bool,i64,i64,datetime[ns],u64,str,str,i64,i64,i64,u8,u8,u8,u8
1,50,0,0,0,12,0,0,0,0,0,0,0,1025,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,14,0,0,0,0,0,0,0,0,1,0,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
2,50,0,0,0,12,0,0,0,0,0,0,0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,15,0,0,0,0,0,0,0,0,1,0,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,3,1,0,12,160,184


In [63]:
unit_born_df.collect_schema()

Schema([('unit_tag_index', UInt32),
        ('unit_tag_recycle', UInt32),
        ('unit_type_name', String),
        ('control_player_id', UInt8),
        ('upkeep_player_id', UInt8),
        ('x', Float32),
        ('y', Float32),
        ('creator_unit_tag_index', UInt32),
        ('creator_unit_tag_recycle', UInt32),
        ('creator_unit_type_name', String),
        ('creator_ability_name', String),
        ('ext_replay_loop', Int64),
        ('ext_replay_seconds', UInt32),
        ('ext_fs_id', UInt64)])

In [64]:
unit_born_df.describe()

statistic,unit_tag_index,unit_tag_recycle,unit_type_name,control_player_id,upkeep_player_id,x,y,creator_unit_tag_index,creator_unit_tag_recycle,creator_unit_type_name,creator_ability_name,ext_replay_loop,ext_replay_seconds,ext_fs_id
str,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64
"""count""",1.17375714e8,1.17375714e8,"""117375714""",1.17375629e8,1.17375629e8,1.17375714e8,1.17375714e8,7.7673455e7,7.7673455e7,"""98104207""","""77673483""",1.17375714e8,1.17375714e8,1.17375714e8
"""null_count""",0.0,0.0,"""0""",85.0,85.0,0.0,0.0,3.9702259e7,3.9702259e7,"""19271507""","""39702231""",0.0,0.0,0.0
"""mean""",492.178509,45.504042,null,1.543063,1.543063,96.055092,92.644661,506.479517,25.172304,null,null,40721.632156,1982.309267,21624.565453
"""std""",288.75173,109.020177,null,1.176864,1.176864,44.437153,43.646477,269.298619,58.1475,null,null,49321.479661,2401.518399,13402.509743
"""min""",0.0,1.0,"""105mmArtillery2""",0.0,0.0,0.0,0.0,1.0,1.0,"""ACPirateCapitalShip""","""""",0.0,0.0,0.0
"""25%""",285.0,2.0,null,1.0,1.0,58.0,57.0,294.0,2.0,null,null,13827.0,673.0,12688.0
"""50%""",460.0,10.0,null,1.0,1.0,97.0,93.0,488.0,4.0,null,null,24687.0,1202.0,15269.0
"""75%""",684.0,42.0,null,2.0,2.0,131.0,126.0,705.0,20.0,null,null,42224.0,2055.0,30632.0
"""max""",3741.0,5169.0,"""zealot_A22""",15.0,15.0,255.0,255.0,3740.0,4487.0,"""uzig""","""toWarPig""",738474.0,35957.0,54323.0


In [65]:
unit_born_df.select(pl.col("creator_unit_type_name").value_counts(sort=True)).collect()
# unit_tag_index and unit_tag_recycle = used to calculate the unique id of a unit
# unit_type_name = name of unit drone, probe, mineralfield, etc.
# control_player_id Either null or 0..15
#                   When null (almost no instances in this data set) could be the campaign, stuff like K5Kerrigan
#                   When 0 it's the system creating mineral fields, geysers, etc.
#                   From 1..15 is the actual users playing.
# upkeep_player_id  Either null or 0..15
#                   As above, when 0, it's the system creating mineral fields, geysers, etc.
#                   When 1..15 actual users playing
# creator_ability_name Larva, Cocoons, OrbitalCommands, Barracks, Nexus, Lair, creating units.

creator_unit_type_name
struct[2]
"{""LurkerMPBurrowed"",43328380}"
"{null,19271507}"
"{""Egg"",16615568}"
"{""Larva"",7402007}"
"{""Hatchery"",6508651}"
"{""BroodLord"",3055107}"
"{""Barracks"",2235507}"
"{""Nexus"",1622251}"
"{""OrbitalCommand"",1530489}"


In [66]:
unit_born_df.filter(
    [
        pl.col("ext_fs_id").eq(chosen_fs_id)
        & ~pl.col("unit_type_name").str.contains("Beacon")
    ]
).join(
    details_lobby_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).with_columns(
        pl.col("user_id").sub(1).alias("control_player_id")
    ),
    on=["control_player_id", "ext_fs_id"],
).collect()

unit_tag_index,unit_tag_recycle,unit_type_name,control_player_id,upkeep_player_id,x,y,creator_unit_tag_index,creator_unit_tag_recycle,creator_unit_type_name,creator_ability_name,ext_replay_loop,ext_replay_seconds,ext_fs_id,player_name,player_toon_region,player_toon_program_id,player_toon_realm,player_toon_id,player_race,player_color_a,player_color_r,player_color_g,player_color_b,player_control,player_team_id,player_observe,player_result,player_working_set_slot_id,player_hero,title,is_blizzard_map,time_utc,time_local_offset,ext_datetime,ext_fs_id_right,ext_fs_sha256,ext_fs_file_name,control,user_id,team_id,observe,working_set_slot_id,map_size_x,map_size_y
u32,u32,str,u8,u8,f32,f32,u32,u32,str,str,i64,u32,u64,str,u8,u32,u32,u64,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,str,bool,i64,i64,datetime[ns],u64,str,str,i64,i64,i64,u8,u8,u8,u8
203,1,"""Hatchery""",1,1,37.0,145.0,null,null,null,null,0,0,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
204,1,"""Larva""",1,1,37.0,142.0,203,1,"""Hatchery""","""""",0,0,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
205,1,"""Larva""",1,1,38.0,142.0,203,1,"""Hatchery""","""""",0,0,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
206,1,"""Larva""",1,1,36.0,142.0,203,1,"""Hatchery""","""""",0,0,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
207,1,"""Drone""",1,1,34.0,148.0,null,null,null,null,0,0,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
208,1,"""Drone""",1,1,34.0,147.0,null,null,null,null,0,0,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
209,1,"""Drone""",1,1,35.0,148.0,null,null,null,null,0,0,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595

In [104]:
(
    unit_born_df.filter(
        [
            pl.col("ext_fs_id").eq(chosen_fs_id)
            & ~pl.col("unit_type_name").str.contains("Beacon")
        ]
    ).join(
        details_lobby_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).with_columns(
            pl.col("user_id").sub(1).alias("control_player_id")
        ),
        on=["control_player_id", "ext_fs_id"],
    )
    .group_by(pl.col("unit_type_name"), pl.col("player_name"))
        .agg(pl.len().alias("freq"))
        .sort(pl.col("freq"), descending=True)
    .collect()
)

unit_type_name,player_name,freq
str,str,u32
"""Larva""","""Serral""",462
"""Egg""","""Serral""",233
"""Drone""","""Serral""",99
"""Roach""","""Serral""",97
"""Probe""","""Classic""",83
"""Stalker""","""Classic""",45
"""Overlord""","""Serral""",26
"""CreepTumorBurrowed""","""Serral""",23
"""Zergling""","""Serral""",18


In [67]:
# System created units, mineral fields, etc.
unit_born_df.filter(
    [
        pl.col("ext_fs_id").eq(chosen_fs_id)
        & pl.col("ext_replay_loop").eq(0)
        & pl.col("control_player_id").eq(0)
    ]
).select(pl.col("unit_type_name").value_counts(sort=True)).collect()

unit_type_name
struct[2]
"{""MineralField"",56}"
"{""MineralField750"",56}"
"{""ProtossVespeneGeyser"",20}"
"{""VespeneGeyser"",6}"
"{""AccelerationZoneLarge"",6}"
"{""MineralField450"",4}"
"{""ShakurasVespeneGeyser"",4}"
"{""DestructibleRockEx16x6"",4}"
"{""RichMineralField750"",3}"


In [68]:
unit_died_df.collect_schema()

Schema([('unit_died_name', String),
        ('unit_tag_index', UInt32),
        ('unit_tag_recycle', UInt32),
        ('killer_player_id', UInt8),
        ('x', UInt8),
        ('y', UInt8),
        ('unit_killer_name', String),
        ('killer_unit_tag_index', UInt32),
        ('killer_unit_tag_recycle', UInt32),
        ('ext_replay_loop', Int64),
        ('ext_replay_seconds', UInt32),
        ('ext_fs_id', UInt64)])

In [68]:
unit_died_df.describe()

Schema([('unit_died_name', String),
        ('unit_tag_index', UInt32),
        ('unit_tag_recycle', UInt32),
        ('killer_player_id', UInt8),
        ('x', UInt8),
        ('y', UInt8),
        ('unit_killer_name', String),
        ('killer_unit_tag_index', UInt32),
        ('killer_unit_tag_recycle', UInt32),
        ('ext_replay_loop', Int64),
        ('ext_replay_seconds', UInt32),
        ('ext_fs_id', UInt64)])

In [70]:
unit_died_df.select(pl.col("unit_killer_name").value_counts(sort=True)).collect()
# unit_died_name name of units dying.
#                Most unit names are InvisibleTargetDummy.
#                It seems when lurker or units A-move, a temporary target_dummy is created at that location and when the unit "attacks" it.
#                Q Are tank shots on InvisibleTargetDummy? There's splash radius/damage somewhere maybe?
# unit_killer_name the killer unit if found.

unit_killer_name
struct[2]
"{""LurkerMPBurrowed"",43716889}"
"{"""",17249356}"
"{""Marine"",1671233}"
"{""Stalker"",1056299}"
"{""Zergling"",1020163}"
"{""SiegeTankSieged"",972513}"
"{""Roach"",735870}"
"{""Hydralisk"",715853}"
"{""Zealot"",701311}"


In [71]:
unit_died_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).join(
    details_lobby_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).with_columns(
        pl.col("user_id").sub(1).alias("killer_player_id")
    ),
    on=["killer_player_id", "ext_fs_id"],
).tail(100).collect()

unit_died_name,unit_tag_index,unit_tag_recycle,killer_player_id,x,y,unit_killer_name,killer_unit_tag_index,killer_unit_tag_recycle,ext_replay_loop,ext_replay_seconds,ext_fs_id,player_name,player_toon_region,player_toon_program_id,player_toon_realm,player_toon_id,player_race,player_color_a,player_color_r,player_color_g,player_color_b,player_control,player_team_id,player_observe,player_result,player_working_set_slot_id,player_hero,title,is_blizzard_map,time_utc,time_local_offset,ext_datetime,ext_fs_id_right,ext_fs_sha256,ext_fs_file_name,control,user_id,team_id,observe,working_set_slot_id,map_size_x,map_size_y
str,u32,u32,u8,u8,u8,str,u32,u32,i64,u32,u64,str,u8,u32,u32,u64,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,str,bool,i64,i64,datetime[ns],u64,str,str,i64,i64,i64,u8,u8,u8,u8
"""Roach""",482,3,2,61,51,"""Adept""",288,4,16869,821,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,3,1,0,12,160,184
"""HighTemplar""",488,1,1,60,51,"""Roach""",436,13,16881,821,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
"""Stalker""",442,10,1,62,59,"""Roach""",423,8,16947,825,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
"""Roach""",512,4,2,66,57,"""Stalker""",535,3,16978,826,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,3,1,0,12,160,184
"""Roach""",548,13,2,67,56,"""Stalker""",535,3,17022,828,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,3,1,0,12,160,184
"""Probe""",413,3,1,62,54,"""Roach""",544,18,17026,828,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,2,0,0,2,160,184
"""Adept""",288,4,1,59,50,"""Roach""",579,1,17027,829,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 

In [105]:
len(unit_died_df.collect())

77779912

In [107]:
(
    unit_died_df
        .filter(pl.col("ext_fs_id").eq(chosen_fs_id))
        .join(
            details_lobby_df
                .filter(pl.col("ext_fs_id").eq(chosen_fs_id))
                .with_columns(pl.col("user_id").sub(1).alias("killer_player_id"))
            ,on=["killer_player_id", "ext_fs_id"],
        )
        .group_by(pl.col("player_name"), pl.col("unit_killer_name"), pl.col("unit_died_name"))
        .agg(pl.len().alias("freq"))
        .sort(pl.col("freq"), descending=True)
        .collect()
)

player_name,unit_killer_name,unit_died_name,freq
str,str,str,u32
"""Classic""","""Stalker""","""Roach""",30
"""Serral""","""Roach""","""Stalker""",27
"""Classic""","""Stalker""","""Zergling""",14
"""Serral""","""Roach""","""Probe""",13
"""Serral""","""Ravager""","""Stalker""",13
"""Classic""","""Adept""","""AdeptPhaseShift""",9
"""Serral""","""Ravager""","""Probe""",7
"""Classic""","""Phoenix""","""Mutalisk""",6
"""Serral""","""""","""MineralField750""",5


In [74]:
cmd_target_point_df.collect_schema()

Schema([('user_id', Int64),
        ('cmd_flags', Int64),
        ('abil_link', UInt16),
        ('abil_cmd_index', Int64),
        ('ability', String),
        ('target_point_x', Int64),
        ('target_point_y', Int64),
        ('target_point_z', Int32),
        ('sequence', Int64),
        ('unit_group', UInt32),
        ('ext_replay_loop', Int64),
        ('ext_replay_seconds', UInt32),
        ('ext_fs_id', UInt64)])

In [75]:
cmd_target_point_df.describe()

statistic,user_id,cmd_flags,abil_link,abil_cmd_index,ability,target_point_x,target_point_y,target_point_z,sequence,unit_group,ext_replay_loop,ext_replay_seconds,ext_fs_id
str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",2.261309e7,2.261309e7,1.0728131e7,1.0728131e7,"""22613090""",2.261309e7,2.261309e7,2.261309e7,2.261309e7,20264.0,2.261309e7,2.261309e7,2.261309e7
"""null_count""",0.0,0.0,1.1884959e7,1.1884959e7,"""0""",0.0,0.0,0.0,0.0,2.2592826e7,0.0,0.0,0.0
"""mean""",2.195515,95032.358634,124.32191,1.019574,null,393013.87234,370429.506006,39136.025229,1684.123698,1.5208e7,3.7335e6,256057.108176,30427.829518
"""std""",2.116589,1.2162e6,190.614978,2.886964,null,159628.419146,155516.530438,595057.118385,748053.148636,1.1209e8,2.9829e8,2.0458e7,14611.257645
"""min""",0.0,256.0,43.0,0.0,"""""",8192.0,8192.0,-1.3832e9,1.0,68923.0,0.0,0.0,0.0
"""25%""",1.0,256.0,46.0,0.0,null,260550.0,241451.0,32744.0,561.0,69930.0,6293.0,431.0,18173.0
"""50%""",2.0,264.0,46.0,0.0,null,391168.0,364161.0,40919.0,1060.0,70037.0,10688.0,733.0,30769.0
"""75%""",3.0,264.0,172.0,0.0,null,522240.0,497664.0,40954.0,1766.0,70133.0,16262.0,1115.0,43093.0
"""max""",26.0,6.2793903e7,12470.0,29.0,"""""",1.048575e6,1.044511e6,149253.0,2.1016e9,3.5588e9,2.7985e10,1.9193e9,54323.0


In [76]:
cmd_target_point_df.select(pl.col("unit_group").value_counts(sort=True)).collect()
# map_elements(lambda x: f'{x:>064b}', return_dtype=pl.String). # Transform i64 to bits.
# cmd_flags is some bit map, not sure about its meaning. Mostly 0s, a few 1s
# unst_abil is a currently unstable feature of s2protocol-rs that tries to generate the ability strings.
# abil_link: mostly null, then followed by 46, 172, 185, 267
# abil_cmd_index: From 0 to 15, mostly nulls, followed by  0, 1,3,15,4,...
# target_point_x, target_point_y and target_point_z are self explanatory, important that the Z is i32 and x,y are i64 at the source.
# sequence: Unknown 12K values, top are: 5,6,4,7,721,681,...
# unit_group: 20 values, most are null, then 70011 with 44 instances, 69957, 70013, but very very few non nulls.

unit_group
struct[2]
"{null,22592826}"
"{69918,1287}"
"{70037,947}"
"{69919,932}"
"{70038,927}"
"{70048,859}"
"{70049,781}"
"{69902,706}"
"{69903,613}"


In [77]:
cmd_target_point_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).join(
    details_lobby_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).with_columns(
        pl.col("user_id").alias("user_id")
    ),
    on=["user_id", "ext_fs_id"],
).sort(by=[pl.col("ext_replay_loop")]).collect()

user_id,cmd_flags,abil_link,abil_cmd_index,ability,target_point_x,target_point_y,target_point_z,sequence,unit_group,ext_replay_loop,ext_replay_seconds,ext_fs_id,player_name,player_toon_region,player_toon_program_id,player_toon_realm,player_toon_id,player_race,player_color_a,player_color_r,player_color_g,player_color_b,player_control,player_team_id,player_observe,player_result,player_working_set_slot_id,player_hero,title,is_blizzard_map,time_utc,time_local_offset,ext_datetime,ext_fs_id_right,ext_fs_sha256,ext_fs_file_name,control,team_id,observe,working_set_slot_id,map_size_x,map_size_y
i64,i64,u16,i64,str,i64,i64,i32,i64,u32,i64,u32,u64,str,u8,u32,u32,u64,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,str,bool,i64,i64,datetime[ns],u64,str,str,i64,i64,u8,u8,u8,u8
2,65800,null,null,"""""",282455,134532,40896,8,null,54,3,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,0,0,2,160,184
3,256,107,0,"""""",263591,119154,40907,5,null,70,4,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,1,0,12,160,184
3,264,null,null,"""""",275872,109184,40907,7,null,224,15,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,1,0,12,160,184
2,264,null,null,"""""",126594,591942,49101,46,null,262,17,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,0,0,2,160,184
2,264,null,null,"""""",176392,589620,49101,51,null,302,20,16594,"""Serral""",2,21298,1,9691277,"""Zerg""",255,180,20,30,2,0,0,1,2,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,0,0,2,160,184
3,264,null,null,"""""",307427,154429,40907,12,null,341,23,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,1,0,12,160,184
3,256,172,1,"""""",286720,147456,40907,27,null,403,27,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Cl

In [86]:
cmd_target_point_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).join(
    details_lobby_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id))
    .filter(pl.col("player_race").eq("Zerg"))
    .select(pl.col(["user_id", "ext_fs_id"])),
    on=["user_id", "ext_fs_id"],
).sort(by=[pl.col("ext_replay_loop")]).select(
    pl.col("abil_link").value_counts(sort=True)
).collect()

abil_link
struct[2]
"{null,192}"
"{46,93}"
"{185,15}"
"{283,14}"
"{267,11}"
"{43,10}"
"{262,9}"
"{108,1}"
"{266,1}"


In [87]:
cmd_target_point_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).join(
    details_lobby_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id))
    .filter(pl.col("player_race").eq("Protoss"))
    .select([pl.col("user_id"), pl.col("ext_fs_id")]),
    on=["user_id", "ext_fs_id"],
).sort(by=[pl.col("ext_replay_loop")]).select(
    pl.col("abil_link").value_counts(sort=True)
).collect()

abil_link
struct[2]
"{null,234}"
"{46,80}"
"{172,49}"
"{216,19}"
"{220,14}"
"{104,13}"
"{178,8}"
"{610,7}"
"{421,4}"


In [80]:
cmd_target_unit_df.collect_schema()

Schema([('user_id', Int64),
        ('cmd_flags', Int64),
        ('abil_link', UInt16),
        ('abil_cmd_index', Int64),
        ('ability', String),
        ('target_unit_flags', UInt16),
        ('tag', UInt32),
        ('snapshot_unit_link', UInt16),
        ('snapshot_control_player_id', Int64),
        ('snapshot_upkeep_player_id', Int64),
        ('snapshot_point_x', Int64),
        ('snapshot_point_y', Int64),
        ('snapshot_point_z', Int32),
        ('sequence', Int64),
        ('ext_replay_loop', Int64),
        ('ext_replay_seconds', UInt32),
        ('ext_fs_id', UInt64)])

In [81]:
cmd_target_unit_df.describe()

statistic,user_id,cmd_flags,abil_link,abil_cmd_index,ability,target_unit_flags,tag,snapshot_unit_link,snapshot_control_player_id,snapshot_upkeep_player_id,snapshot_point_x,snapshot_point_y,snapshot_point_z,sequence,ext_replay_loop,ext_replay_seconds,ext_fs_id
str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",7.125606e6,7.125606e6,3.475483e6,3.475483e6,"""7125606""",7.125606e6,7.125606e6,7.125606e6,7.125603e6,7.125603e6,7.125606e6,7.125606e6,7.125606e6,7.125606e6,7.125606e6,7.125606e6,7.125606e6
"""null_count""",0.0,0.0,3.650123e6,3.650123e6,"""0""",0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",2.086311,2654.07215,186.680315,0.288126,null,113.066611,6.0949e7,374.081552,0.934703,0.934695,391232.314899,371006.088755,43210.26402,1343.051647,1.7985e6,123346.371257,29839.433946
"""std""",2.103183,268920.44046,234.444969,0.715471,null,18.19816,4.8034e7,367.5903,0.973829,0.973832,191502.086512,186515.401273,590323.720821,465040.784906,2.1840e8,1.4978e7,14685.755854
"""min""",0.0,256.0,43.0,0.0,"""""",111.0,0.0,23.0,0.0,0.0,11177.0,8192.0,-1.0022e9,1.0,0.0,0.0,0.0
"""25%""",0.0,256.0,46.0,0.0,null,111.0,2.3592961e7,97.0,0.0,0.0,212992.0,202752.0,40912.0,353.0,4541.0,311.0,16909.0
"""50%""",1.0,264.0,113.0,0.0,null,111.0,5.6098817e7,133.0,1.0,1.0,390676.0,362496.0,40944.0,810.0,8546.0,586.0,29962.0
"""75%""",3.0,264.0,173.0,0.0,null,111.0,8.4410369e7,547.0,2.0,2.0,563200.0,541980.0,49120.0,1469.0,13855.0,950.0,42546.0
"""max""",20.0,5.201338e7,29135.0,27.0,"""""",28036.0,1.4899e9,28090.0,15.0,15.0,1.032192e6,1.040383e6,162816.0,9.02046459e8,2.9723e10,2.0385e9,54323.0


In [92]:
cmd_target_unit_df.select(pl.col("ability").value_counts(sort=True)).collect()
# User-ID is 0-indexed
# cmd_flags 8 values, seems bitmap
# abil_link: mostly null, then followed by 46, 172, 185, 267
# abil_cmd_index: 7 possible values, mostly nulls, followed by  0,2,1,3,23,18
# target_unit_flags can be either 111 or 175 I guess some bitmap
# tag: The tag_index+tag_recycle combo
# snapshot_unit_link 321 possible values, top are 545,109,903,111,547,... Maybe related to abil_link?
# snapshot_control_player_id from 0 to 15
# snapshot_upkeep_player_id 0 to 15
# snapshot_pointx, y, z positions, again z is i32.
# sequence 9K values, starts at 2, 1, 6, 3,66, 82,68 maybe bitmap

ability
struct[2]
"{"""",7078708}"
"{""attack.Execute"",11040}"
"{""SpawnLarva.Execute"",10021}"
"{""ChronoBoostEnergyCost.Execute"",6500}"
"{""SCVHarvest.Gather"",6075}"
"{""Transfusion.Execute"",1875}"
"{""DroneHarvest.Gather"",1703}"
"{""CalldownMULE.Execute"",1554}"
"{""RallyNexus.Rally1"",1228}"


In [94]:
cmd_target_unit_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).join(
    details_lobby_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).with_columns(
        pl.col("user_id").alias("user_id")
    ),
    on=["user_id", "ext_fs_id"],
).filter(pl.col("ability") != "").sort(by=[pl.col("ext_replay_loop")]).collect()

user_id,cmd_flags,abil_link,abil_cmd_index,ability,target_unit_flags,tag,snapshot_unit_link,snapshot_control_player_id,snapshot_upkeep_player_id,snapshot_point_x,snapshot_point_y,snapshot_point_z,sequence,ext_replay_loop,ext_replay_seconds,ext_fs_id,player_name,player_toon_region,player_toon_program_id,player_toon_realm,player_toon_id,player_race,player_color_a,player_color_r,player_color_g,player_color_b,player_control,player_team_id,player_observe,player_result,player_working_set_slot_id,player_hero,title,is_blizzard_map,time_utc,time_local_offset,ext_datetime,ext_fs_id_right,ext_fs_sha256,ext_fs_file_name,control,team_id,observe,working_set_slot_id,map_size_x,map_size_y
i64,i64,u16,i64,str,u16,u32,u16,i64,i64,i64,i64,i32,i64,i64,u32,u64,str,u8,u32,u32,u64,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,str,bool,i64,i64,datetime[ns],u64,str,str,i64,i64,u8,u8,u8,u8
3,256,107,0,"""RallyNexus.Rally1""",111,40632321,545,0,0,126976,141312,49088,9,240,16,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,1,0,12,160,184
3,256,107,0,"""RallyNexus.Rally1""",111,31457281,545,0,0,155648,116736,49088,11,316,21,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,1,0,12,160,184
3,256,107,0,"""RallyNexus.Rally1""",111,31457281,545,0,0,155648,116736,49088,31,478,32,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,1,0,12,160,184
3,256,107,0,"""RallyNexus.Rally1""",111,3670017,547,0,0,151552,112640,49088,42,721,49,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,1,0,12,160,184
3,256,107,0,"""RallyNexus.Rally1""",111,3670017,547,0,0,151552,112640,49088,46,784,53,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,1,0,12,160,184
3,256,723,0,"""ChronoBoostEnergyCost.Execute""",111,57671681,82,2,2,153600,141312,49088,54,837,57,16594,"""Classic""",2,21298,1,9691041,"""Protoss""",255,0,66,255,2,1,0,2,12,"""""","""Persephone LE""",true,133979343142010758,108000000000,2025-07-25 13:25:14.201075,16594,"""3f32defc30edd6fd9275ca96dfa5b5a3a2c3b8697c6540e865e2ef595c0265a3""","""/home/seb/SCReplaysOnNVMe/2025EWCReplayPack/3 - Playoffs/8 - Grand Finals - Serral vs Classic/20250725 - Game1 - Classic vs Serral - Persephone.SC2Replay""",2,1,0,12,160,184
3,256,107,0,"""RallyNexus.Rally1""",111,23592961,1179,0,0,169984,112640,49088,

In [102]:
cmd_target_unit_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).join(
    details_lobby_df.filter(pl.col("ext_fs_id").eq(chosen_fs_id)).with_columns(
        pl.col("user_id").alias("user_id")
    ),
    on=["user_id", "ext_fs_id"],
).filter(pl.col("ability") != "").group_by(pl.col("ability"), pl.col("player_name")).agg(pl.len().alias("freq")).sort(pl.col("freq"),descending=True).collect()

ability,player_name,freq
str,str,u32
"""SpawnLarva.Execute""","""Serral""",20
"""RallyNexus.Rally1""","""Classic""",17
"""DroneHarvest.Gather""","""Serral""",12
"""Transfusion.Execute""","""Serral""",8
"""attack.Execute""","""Serral""",7
"""ChronoBoostEnergyCost.Execute""","""Classic""",7
"""attack.Execute""","""Classic""",5
"""RallyHatchery.Rally2""","""Serral""",2
"""Rally.Rally1""","""Classic""",2
